Задача: научиться находить в тексте лексически/стилистически неправильные сочетания (коллокации) и предлагать более правильные замены.

Решение: задача была разделена на подзадачи - 1) поиск неправильных сочетаний и 2) поиск возможных замен по эталонному списку, который был взят из корпуса русских академических текстов (корпус тоже делали мы). Поскольку большинство ошибок встречаются в именных и глагольных словосочетаниях, работать будем только с ними. 

Эталонные списки делаются из списков коллокаций корпуса, которые тоже делала я. Код можно посмотреть тут: https://github.com/MariaFjodorowa/catandthekittens/blob/develop/collocation_frequences/collocations%202_1.ipynb, готовые списки здесь: https://drive.google.com/drive/folders/1k_N-DZ-nLL5ro66-LxIaE4-dRwirdwZh

In [2]:
import pandas as pd
import re
import nltk
import gensim
from gensim.models import Word2Vec
from natasha import NamesExtractor, AddressExtractor, DatesExtractor
import pymorphy2
from sklearn.externals import joblib

model = gensim.models.KeyedVectors.load_word2vec_format('ruwikiruscorpora_upos_skipgram_300_2_2018.vec.gz', binary=False)
morph = pymorphy2.MorphAnalyzer()
domain_model=joblib.load('domain_model.pkl')

Сейчас я использую модель из русвекторес, для работы с которой надо, во-первых, лемматизировать слова, во-вторых, прилеплять к ним теги. На самом деле у нашего проекта есть свои собственные вордтувеки, посчитанные на корпусе киберленинки, для которых не надо преобразовывать слова и которые нельзя заставить работать у меня на виртуальной машине (зато они прекрасно функционируют у моих коллег).

In [3]:
def convert_pm(word, tag):
    if tag=='S':
        return morph.parse(word)[0].normal_form+'_'+'NOUN'
    elif tag=='V':
        return morph.parse(word)[0].normal_form+'_'+'VERB'

In [5]:
def tag(word):
    p=str(morph.parse(word)[0].tag).split(",")[0]
    if p=='NOUN':
        return 'S'
    elif p=='VERB':
        return 'V'
    else:
        return 'X'

В зависимости от того, к какому домену относится текст, выбираем эталонный список коллокаций для сравнения. Для определения домена используется SVM, предобученная на шестиграммах из каждого домена (по 100000 на домен). Код этой модели также могу показать, но там ничего особенного. F-мера модели - 94.

In [6]:
def domain(string):
    domain=domain_model.predict(list(string))[0]
    if domain=='hist':
        df=pd.read_csv('suggestions_hist.csv')
        return df
    elif domain=='ling':
        df=pd.read_csv('suggestions.csv')
        return df
    elif domain=='law':
        df=pd.read_csv('suggestions_law.csv')
        return df
    elif domain=='pol':
        df=pd.read_csv('suggestions_pol.csv')
        return df
    elif domain=='psy':
        df=pd.read_csv('suggestions_psy.csv')
        return df
    elif domain=='ec':
        df=pd.read_csv('suggestions_ec.csv')
        return df
    elif domain=='soc':
        df=pd.read_csv('suggestions_soc.csv')
        return df

Для начала в тексте находятся коллокации, которые подходят нам по тегам и которых нет в эталонном списке. Попытки изобрести более совершенный способ искать ошибки предпринимались, но успешны не были, поэтому пока что применяется словарный способ. Имена собственные и подобные слова исключаются из поиска.

In [7]:
def search(text, collocations):
    first=list(collocations.first_word)
    second=list(collocations.second_word)
    candidates={}
    extractors = [NamesExtractor(), AddressExtractor(), DatesExtractor()]
    personal=[]
    for extractor in extractors:
        matches = extractor(text)
        for match in matches:
            start, stop = match.span
            for i in re.findall('\w+', text[start:stop]):
                personal.append(i.lower())
    string=re.findall('[а-яА-ЯЁё]+', text.lower())
    bigrams=list(nltk.bigrams(string))
    for bigram in bigrams:
        w1,w2=bigram[0], bigram[1]
        if w1 not in personal and w2 not in personal:
            tag1=tag(w1)
            tag2=tag(w2)
            if (tag1=='S' or tag1=='V') and (tag2=='S' or tag2=='V'):
                if w1 in first or w2 in second:
                    if w1 in first and w2 in second:
                        indices1 = [i for i, x in enumerate(first) if x == w1]
                        indices2 = [i for i, x in enumerate(second) if x == w2]
                        if not set(indices1).isdisjoint(indices2):
                            pass
                        else:
                            candidates['{} {}'.format(w1, w2)]='{} {}'.format(tag1, tag2)
                    else:
                        candidates['{} {}'.format(w1, w2)]='{} {}'.format(tag1, tag2)
    return candidates

Кандидаты на замену подбираются по эталонному списку. Вес кандидата определяется, исходя из:  
1) PMI между правильным словом и словом на замену (посчитан заранее и отмечен в списках);  
2) схожести, вычисленной при помощи вордтувека;  
3) близости слов в коллокационном кластере. Эта метрика вычисляется, если оба слова есть в эталонном списке по отдельности, но не вместе. Допустим, есть пара слов N и V, у которых в эталонном списке есть следующие коллокаты: N(V11, V12, V13), V(N11, N12, N13). У данных коллокатов есть свои пары V11(N21, N22), V12(N23, N24) и т.д. Мера близости N и V по кластеру - это количество таких слов N21, N22,..., которые совпадают с коллокатами V(N11, N12, N13), и таких слов V21, V22,..., которые совпадают с коллокатами N(V11, V12, V13);  
4) схожести тегов заменяемого слова и замены.  

Подробнее об этом алгоритме: https://aclanthology.info/pdf/W/W09/W09-2107.pdf

In [8]:
def suggest(candidates, collocations):
    c_max=max(collocations.pmi)
    corrections={}
    for item in candidates.items():
        suggestions={}
        w1=item[0].split()[0]
        w2=item[0].split()[1]
        tag1=item[1].split()[0]
        tag2=item[1].split()[1]
        w1_sub=[]
        w2_sub=[]
        for i in range(len(collocations)-1):
            #assuming that second word is wrong
            if collocations.first_word[i]==w1 and collocations.second_tag[i]==tag2:
                suggestions[w1+ ' ' +collocations.second_word[i]]=collocations.pmi[i]/c_max
                try:
                    suggestions[collocations.first_word[i]+ ' ' +w2]+=model.similarity(convert_pm(w1, tag1), convert_pm(collocations.second_word[i], tag2))
                except:
                    pass
                if str(morph.parse(w2)[0].tag)==str(morph.parse(collocations.second_word[i])[0].tag):
                    suggestions[w1+ ' ' +collocations.second_word[i]]+=0.1
                w2_sub.append(collocations.second_word[i])
            #assuming that first word is wrong
            if collocations.second_word[i]==w2 and collocations.first_tag[i]==tag1:
                suggestions[collocations.first_word[i]+ ' ' +w2]=collocations.pmi[i]/c_max
                try:
                    suggestions[collocations.first_word[i]+ ' ' +w2]+=model.similarity(convert_pm(collocations.first_word[i],tag1), convert_pm(w2,tag2))
                except:
                    pass
                if str(morph.parse(w1)[0].tag)==str(morph.parse(collocations.first_word[i])[0].tag):
                    suggestions[collocations.first_word[i]+ ' ' +w2]+=0.1
                w1_sub.append(collocations.first_word[i])
        #collocation cluster if both words could be substituted
        if w1_sub and w2_sub:
            for w in w1_sub:
                for j in range(len(collocations)-1):
                    if collocations.first_word[j]==w and collocations.second_word[j] in w2_sub:
                        suggestions[w+' '+w2]+=1.0/len(w2_sub)
            for k in w2_sub:
                for j in range(len(collocations)-1):
                    if collocations.second_word[j]==k and collocations.first_word[j] in w1_sub:
                        suggestions[w1+' '+k]+=1.0/len(w1_sub)
        variants=[]
        for i in sorted(suggestions.items(), key=lambda x: x[-1], reverse=True):
            if i[1]>=0.5:
                variants.append(i)
        corrections[item[0]]=variants[:10]
    return corrections

Протестируем несколько строчек.  
Первая строка: спорное сочетание - "разрешение вопроса";  
Вторая строка: правильная (не берём в рассчёт пунктуацию :);  
Третья строка: неправильное сочетание - "автор думает".  

In [9]:
text1='цель работы - разрешение вопроса об использовании такого подхода в исследованиях'

text2='результаты показанные на данном примере'

text3='автор думает, что данный пример продемонстрировал'

Здесь, кроме спорного сочетания, выловлено сочетание, являющееся правильным, что выявляет необходимость доработки системы отбора кандидатов - в частности, добавление элементов, которые бы не позволили захватывать слова на границах именных и глагольных групп (например, "работы разрешение" в данном случае). Коллокации, предлагаемые на замену таких выхваченных групп, бывают немного странными.

In [10]:
suggest(search(text1, domain(text1)), domain(text1))

/home/stee/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


{'работы разрешение': [('работы снегурочкой', 0.6445432209029189),
  ('работы упрощение', 0.6445432209029189),
  ('работы механизмов', 0.5186245552809781)],
 'разрешение вопроса': [('обсуждение вопроса', 0.9807323023820061),
  ('историей вопроса', 0.8790489815848778),
  ('актами вопроса', 0.870984195507897),
  ('решении вопроса', 0.838838947300385),
  ('разрешение вере', 0.8055867038305552),
  ('изложение вопроса', 0.8003963391133424),
  ('контексте вопроса', 0.75713387700819),
  ('постановка вопроса', 0.7283893072731861),
  ('анализе вопроса', 0.7278821543731434),
  ('признак вопроса', 0.7197526696583064)]}

В данном случае предложение правильное и замен предложено не было.

In [11]:
suggest(search(text2, domain(text2)), domain(text2))

/home/stee/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


{}

Здесь предложено достаточно много подходящих вариантов на замену стилистически неверного сочетания "автор думает". Выхвачено также сочетание "пример продемонстрировал", правильное в целом, но несколько нетипичное, т.к. чаще используется "продемонстрировать на примере". Для него предложено намного меньше замен.

In [12]:
suggest(search(text3, domain(text3)), domain(text3))

/home/stee/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


{'автор думает': [('автор наталкивается', 0.7601508028266003),
  ('автор придерживается', 0.6741979416716583),
  ('автор апеллирует', 0.6601508028266003),
  ('автор посвящает', 0.6601508028266003),
  ('автор сосредоточивает', 0.6601508028266003),
  ('автор стремился', 0.6601508028266003),
  ('автор обращается', 0.6342321372046617),
  ('автор публикует', 0.605920585319349),
  ('автор утверждает', 0.5725450736964377),
  ('автор выражает', 0.5659547808523554)],
 'пример продемонстрировал': [('пример совместим', 0.5625742287685846),
  ('пример подтверждает', 0.5083440112613357)]}

В целом можно видеть, что большинство неточностей возникают из-за несовершенств системы поиска неправильных сочетаний, над которой я и планирую работать дальше.